# Imports

In [1]:
import math
import pandas as pd
import numpy as np
import glob
import datetime
pd.options.mode.chained_assignment = None

# Table of Contents
 <p><div class="lev1"><a href="#Task-1.-Compiling-Ebola-Data"><span class="toc-item-num">Task 1.&nbsp;&nbsp;</span>Compiling Ebola Data</a></div>
 <div class="lev1"><a href="#Task-2.-RNA-Sequences"><span class="toc-item-num">Task 2.&nbsp;&nbsp;</span>RNA Sequences</a></div>
 <div class="lev1"><a href="#Task-3.-Class-War-in-Titanic"><span class="toc-item-num">Task 3.&nbsp;&nbsp;</span>Class War in Titanic</a></div></p>

In [2]:
DATA_FOLDER = 'Data' # Use the data folder provided in Tutorial 02 - Intro to Pandas.

## Task 1. Compiling Ebola Data

The `DATA_FOLDER/ebola` folder contains summarized reports of Ebola cases from three countries (Guinea, Liberia and Sierra Leone) during the recent outbreak of the disease in West Africa. For each country, there are daily reports that contain various information about the outbreak in several cities in each country.

Use pandas to import these data files into a single `Dataframe`.
Using this `DataFrame`, calculate for *each country*, the *daily average per month* of *new cases* and *deaths*.
Make sure you handle all the different expressions for *new cases* and *deaths* that are used in the reports.

### Constants

In [3]:
DEFAULT_NAME_COLUMN_DATE = 'Date'
DEFAULT_NAME_COLUMN_DESCRIPTION = 'Description'
DEFAULT_NAME_COLUMN_TOTAL = 'Totals'
DEFAULT_NAME_COLUMN_COUNTRY = 'Country'

DEFAULT_NAME_ROW_DESCRIPTION_NEW_CASES = 'New cases'
DEFAULT_NAME_ROW_DESCRIPTION_NEW_DEATHS = 'New deaths'

DEFAULT_DATA_FORMAT = '%Y-%m-%d'

### Auxiliary functions

In [4]:
def sanitize_date(date, original_formats):
    '''
    Transform a date specified in a string with a certain original
    format into a string with the date in the default format.
    '''
    date_rep = None
    try:
        date_rep = datetime.datetime.strptime(date, original_formats[0])
    except ValueError:
        date_rep = datetime.datetime.strptime(date, original_formats[1])
    return date_rep.strftime(DEFAULT_DATA_FORMAT)

In [5]:
def estimate_daily_new_deaths(np_list):
    '''
    Given a list of accumulated data, calculate the data for each day.
    It assumes ordered data.
    '''
    last_known_n = -1
    final_list = list()
    for i, elem in np_list.iteritems():
        if (math.isnan(elem)):
            final_list.append(0)
        else:
            if (last_known_n == -1):
                final_list.append(0)
            else:
                final_list.append(elem - last_known_n)
            last_known_n = elem
    return np.array(final_list)

In [6]:
country_keys = ['guinea', 'liberia', 'sl']
paths = {x: DATA_FOLDER + '/ebola/'+ x + '_data' for x in country_keys}
countries = {'guinea': 'Guinea', 'liberia': 'Liberia', 'sl': 'Sierra Leone'}

wanted_columns = {'guinea': ['Date', 'Description', 'Totals'],
                      'liberia': ['Date', 'Variable', 'National'],
                      'sl': ['date', 'variable', 'National']}

date_original_formats = {'guinea': ['%Y-%m-%d', '%y-%m-%d'],
                         'liberia': ['%m/%d/%Y', '%m/%d/%y'],
                         'sl': ['%Y-%m-%d', '%y-%m-%d']}
dataframe_list = {}

### Read files

In [7]:
for i in country_keys:
    temp_list = []
    all_files = glob.glob(paths[i] + '/*.csv')
    for file in all_files:
        temp_df = pd.read_csv(file)
        temp_list.append(temp_df)
    dataframe_list[i] = pd.concat(temp_list)
    
    # Add 'Country' column to dataframe
    #dataframe_list[i][DEFAULT_NAME_COLUMN_COUNTRY] = countries[i]
    
    # Format dates and order dataframes by date
    dataframe_list[i][wanted_columns[i][0]] = np.array([sanitize_date(x, date_original_formats[i])
                                                        for x in dataframe_list[i][wanted_columns[i][0]]])
    
    dataframe_list[i][wanted_columns[i][0]] = pd.to_datetime(dataframe_list[i][wanted_columns[i][0]])
    dataframe_list[i].sort_values(by=wanted_columns[i][0])

### Parsing functions

In [8]:
def parse_guinea_files():
    # Select certain column from the CSV formatted files
    parsed_guinea_df = dataframe_list['guinea'][wanted_columns['guinea']]
    wanted_row_list = ['Total new cases registered so far',
                       'New deaths registered', 
                       'New deaths registered today']
    
    # Select all the rows that match desired names for 'Description' column
    parsed_guinea_df = parsed_guinea_df[parsed_guinea_df['Description'].isin(wanted_row_list)]
    
    # Rename every row value for 'Description' column 
    parsed_guinea_df.loc[parsed_guinea_df['Description'] ==
                         wanted_row_list[0], 'Description'] = DEFAULT_NAME_ROW_DESCRIPTION_NEW_CASES
    
    parsed_guinea_df.loc[parsed_guinea_df['Description'] ==
                         wanted_row_list[1], 'Description'] = DEFAULT_NAME_ROW_DESCRIPTION_NEW_DEATHS
    
    parsed_guinea_df.loc[parsed_guinea_df['Description'] ==
                         wanted_row_list[2], 'Description'] = DEFAULT_NAME_ROW_DESCRIPTION_NEW_DEATHS
    
    # Add 'Country' column to dataframe
    parsed_guinea_df[DEFAULT_NAME_COLUMN_COUNTRY] = countries['guinea']
    
    return parsed_guinea_df

In [9]:
def parse_liberia_files():
    # Select certain columns from the CSV formatted files
    liberia_df = dataframe_list['liberia'][wanted_columns['liberia']]

    wanted_row_list = ['New Case/s (Suspected)',
                       'New Case/s (Probable)',
                       'New case/s (confirmed)',
                       'Newly reported deaths']

    # Create temporary dataframe for new case entries
    # (with every row that matches the desired 'Variables' column value)
    liberia_df_new_cases = liberia_df[liberia_df['Variable'].isin(wanted_row_list[0:3])]

    # Sum all of the values for 'Probable', 'Variable' and 'Confirmed' new cases
    liberia_df_new_cases = liberia_df_new_cases.groupby('Date').sum()

    liberia_df_new_cases['Date'] = liberia_df_new_cases.index
    liberia_df_new_cases['Variable'] = DEFAULT_NAME_ROW_DESCRIPTION_NEW_CASES
    
    # Create temporary dataframe for new deaths entries
    liberia_df_new_deaths = liberia_df[liberia_df['Variable'] == wanted_row_list[3]]
    liberia_df_new_deaths['Variable'] = DEFAULT_NAME_ROW_DESCRIPTION_NEW_DEATHS
    
    # Create new dataframe with new deaths and cases (concatenating the two temporary)
    parsed_liberia_df = pd.concat([liberia_df_new_cases, liberia_df_new_deaths])
    
    parsed_liberia_df.rename(columns={'Date':DEFAULT_NAME_COLUMN_DATE,
                                      'Variable':DEFAULT_NAME_COLUMN_DESCRIPTION,
                                      'National':DEFAULT_NAME_COLUMN_TOTAL}, inplace=True)

    # Add 'Country' column to dataframe
    parsed_liberia_df[DEFAULT_NAME_COLUMN_COUNTRY] = countries['liberia']
    
    return parsed_liberia_df

In [10]:
def parse_sierra_files():
    # Select certain columns from the CSV formatted files
    parsed_sierra_df = dataframe_list['sl'][wanted_columns['sl']]
    
    wanted_row_list = ['new_noncase', 'new_suspected', 'new_probable', 'new_confirmed',
                       'death_confirmed', 'death_probable', 'death_suspected']
    
    # Create temporary dataframe for new case entries
    # (with every row that matches the desired 'variables' column value)
    sierra_df_new_cases = parsed_sierra_df[parsed_sierra_df['variable'].isin(wanted_row_list[0:4])]
    
    # Sum all of the values for 'Probable', 'Variable', 'Confirmed' and 'Noncase' new cases
    sierra_df_new_cases = sierra_df_new_cases.groupby('date')['National'].apply(
        lambda x: np.array([float(y) for y in x]).sum()).to_frame()
    
    sierra_df_new_cases['date'] = sierra_df_new_cases.index
    sierra_df_new_cases['variable'] = DEFAULT_NAME_ROW_DESCRIPTION_NEW_CASES
    
    # Create temporary dataframe for new deaths' accumulator entries
    sierra_df_new_deaths_cum = parsed_sierra_df[parsed_sierra_df['variable'].isin(wanted_row_list[4:7])]
    
    # Sum all of the values for 'Probable', 'Suspected' and 'Confirmed' new death accumulators
    sierra_df_new_deaths_cum = sierra_df_new_deaths_cum.groupby('date')['National'].apply(
        lambda x: np.array([float(y) for y in x]).sum()).to_frame()
    
    sierra_df_new_deaths_cum['date'] = sierra_df_new_deaths_cum.index
    sierra_df_new_deaths_cum['variable'] = 'New deaths accumulator'
    
    # 
    sierra_df_new_deaths_cum['National'] = estimate_daily_new_deaths(sierra_df_new_deaths_cum['National'])
    sierra_df_new_deaths_cum['variable'] = DEFAULT_NAME_ROW_DESCRIPTION_NEW_DEATHS
    
    #
    parsed_sierra_df = pd.concat([sierra_df_new_cases, sierra_df_new_deaths_cum])
    
    parsed_sierra_df.rename(columns={'date':DEFAULT_NAME_COLUMN_DATE,
                                     'variable':DEFAULT_NAME_COLUMN_DESCRIPTION,
                                     'National':DEFAULT_NAME_COLUMN_TOTAL}, inplace=True)
    
    # Add 'Country' column to dataframe
    parsed_sierra_df[DEFAULT_NAME_COLUMN_COUNTRY] = countries['sl']
    
    return parsed_sierra_df

In [25]:
complete_df = pd.concat([parse_guinea_files(), parse_liberia_files(), parse_sierra_files()])
complete_df.reset_index(inplace=True, drop=True)

In [26]:
print(complete_df.isnull().values.any())
complete_df[complete_df['Totals'].isnull()]

True


,Country,Date,Description,Totals
137,Liberia,2014-12-03,New cases,NaN
146,Liberia,2014-12-07,New deaths,NaN
154,Liberia,2014-12-03,New deaths,NaN
164,Liberia,2014-09-20,New deaths,NaN
171,Liberia,2014-12-05,New deaths,NaN
173,Liberia,2014-12-02,New deaths,NaN
186,Liberia,2014-11-28,New deaths,NaN
193,Liberia,2014-12-01,New deaths,NaN
202,Liberia,2014-12-09,New deaths,NaN
206,Liberia,2014-12-04,New deaths,NaN


# Task 2. RNA Sequences

In the `DATA_FOLDER/microbiome` subdirectory, there are 9 spreadsheets of microbiome data that was acquired from high-throughput RNA sequencing procedures, along with a 10<sup>th</sup> file that describes the content of each. 

Use pandas to import the first 9 spreadsheets into a single `DataFrame`.
Then, add the metadata information from the 10<sup>th</sup> spreadsheet as columns in the combined `DataFrame`.
Make sure that the final `DataFrame` has a unique index and all the `NaN` values have been replaced by the tag `unknown`.

In [5]:
# Write your answer here

## Task 3. Class War in Titanic

Use pandas to import the data file `Data/titanic.xls`. It contains data on all the passengers that travelled on the Titanic.

In [ ]:
from IPython.core.display import HTML
HTML(filename=DATA_FOLDER+'/titanic.html')

For each of the following questions state clearly your assumptions and discuss your findings:
1. Describe the *type* and the *value range* of each attribute. Indicate and transform the attributes that can be `Categorical`. 
2. Plot histograms for the *travel class*, *embarkation port*, *sex* and *age* attributes. For the latter one, use *discrete decade intervals*. 
3. Calculate the proportion of passengers by *cabin floor*. Present your results in a *pie chart*.
4. For each *travel class*, calculate the proportion of the passengers that survived. Present your results in *pie charts*.
5. Calculate the proportion of the passengers that survived by *travel class* and *sex*. Present your results in *a single histogram*.
6. Create 2 equally populated *age categories* and calculate survival proportions by *age category*, *travel class* and *sex*. Present your results in a `DataFrame` with unique index.

In [ ]:
# Write your answer here